In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 

import warnings
warnings.filterwarnings("ignore")

from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import LabelEncoder 

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier




In [22]:
df1 = pd.read_excel('case_study1.xlsx')

df2 = pd.read_excel('case_study2.xlsx')

In [23]:
print(df1.shape)
print(df2.shape)

(51336, 26)
(51336, 62)


In [24]:
# remove nulls

df1 = df1[df1['Age_Newest_TL'] != -99999]

df1.shape

(51296, 26)

In [25]:
cols_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        cols_removed.append(i)

cols_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [26]:
# remove cols with many null values

df2 = df2.drop(cols_removed, axis=1)

df2.shape

(51336, 54)

In [27]:
# remove null rows

for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

df2.shape
    

(42066, 54)

In [28]:
# checking commom columns

for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [29]:
# join both datasets

df = df1.merge(df2, how='inner', on='PROSPECTID')

df.shape

(42064, 79)

In [30]:
# check categorical columns

for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [31]:
# chi-square test 

for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, "---", pval)

MARITALSTATUS --- 3.5781808610388605e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.9079361001865664e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.849976105554191e-287


In [32]:
# all values less than equal to 0.05, will accept all 

In [33]:
# check numerical columns

num_cols = []

for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID', 'Approved_Flag']:
        num_cols.append(i)

num_cols

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [34]:
# VIF sequentially check

vif_data = df[num_cols]
total_cols = vif_data.shape[1]
cols_to_keep = []
col_index = 0

for i in range(0, total_cols):
    vif_value = variance_inflation_factor(vif_data, col_index)
    print(col_index, "---", vif_value)

    if vif_value <= 6:
        cols_to_keep.append(num_cols[i])
        col_index = col_index + 1

    else: 
        vif_data = vif_data.drop(num_cols[i], axis=1)


0 --- inf
0 --- inf
0 --- 11.320180023967982
0 --- 8.36369803500036
0 --- 6.5206478777909425
0 --- 5.14950161821261
1 --- 2.611111040579735
2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477212
2 --- 3.832800792153082
3 --- 6.0996533816466405
3 --- 5.581352009642814
4 --- 1.9855843530987702
5 --- inf
5 --- 4.809538302819332
6 --- 23.270628983464636
6 --- 30.595522588099946
6 --- 4.384346405965575
7 --- 3.0646584155234122
8 --- 2.898639771299225
9 --- 4.377876915347337
10 --- 2.2078535836958486
11 --- 4.916914200506877
12 --- 5.214702030064743
13 --- 3.3861625024231516
14 --- 7.84058330947899
14 --- 5.255034641721459
15 --- inf
15 --- 7.380634506427207
15 --- 1.421005001517572
16 --- 8.083255010190301
16 --- 1.6241227524040012
17 --- 7.257811920140015
17 --- 15.596243832683006
17 --- 1.825857047132431
18 --- 1.5080839450032724
19 --- 2.1720888348245815
20 --- 2.6233975535272367
21 --- 2.2959970812106216
22 --- 7.360578319196457
22 --- 2.1602387773102514
23 --- 2.8686288267891493
24

In [36]:
len(cols_to_keep)

39

In [37]:
# check anova

cols_to_keep_num = []

for i in cols_to_keep:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']

    f_statistics, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        cols_to_keep_num.append(i)


len(cols_to_keep_num)

37

In [38]:
# listing all the final features

features = cols_to_keep_num + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [39]:
print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique())
print(df['GENDER'].unique())
print(df['last_prod_enq2'].unique())
print(df['first_prod_enq2'].unique())


['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [40]:
# ordinal encoding for education

df.loc[df['EDUCATION']=='SSC', ['EDUCATION']] = 1
df.loc[df['EDUCATION']=='12TH', ['EDUCATION']] = 2
df.loc[df['EDUCATION']=='UNDER GRADUATE', ['EDUCATION']] = 3
df.loc[df['EDUCATION']=='GRADUATE', ['EDUCATION']] = 3
df.loc[df['EDUCATION']=='POST-GRADUATE', ['EDUCATION']] = 4
df.loc[df['EDUCATION']=='PROFESSIONAL', ['EDUCATION']] = 3
df.loc[df['EDUCATION']=='OTHERS', ['EDUCATION']] = 1

df['EDUCATION'] = df['EDUCATION'].astype(int)
df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [41]:
# one-hot encoding for other categorical columns

df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'])
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [42]:
# random forest

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

x_trian, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

rf_classifier.fit(x_trian, y_train)

y_pred = rf_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()



Accuracy = 0.7657197194817544

Class p1
Precision = 0.8329466357308585
Recall = 0.7080867850098619
F1_score = 0.7654584221748401

Class p2
Precision = 0.7975805077525984
Recall = 0.9278493557978196
F1_score = 0.8577973245372915

Class p3
Precision = 0.44976816074188564
Recall = 0.21962264150943397
F1_score = 0.295131845841785

Class p4
Precision = 0.7265700483091787
Recall = 0.7308066083576288
F1_score = 0.7286821705426356



In [44]:
# xgboost

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_trian, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

xgb_classifier.fit(x_trian, y_train)

y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()


Accuracy = 0.7783192677998336

Class p1
Precision = 0.823906083244397
Recall = 0.7613412228796844
F1_score = 0.7913890312660175

Class p2
Precision = 0.8255418233924413
Recall = 0.913577799801784
F1_score = 0.8673315769665035

Class p3
Precision = 0.4756380510440835
Recall = 0.30943396226415093
F1_score = 0.37494284407864653

Class p4
Precision = 0.7342386032977691
Recall = 0.7356656948493683
F1_score = 0.7349514563106796



In [46]:
# decision tree

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

x_trian, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)

dt_model.fit(x_trian, y_train)

y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()


Accuracy = 0.7103292523475574

Class p1
Precision = 0.7246804326450345
Recall = 0.7268244575936884
F1_score = 0.7257508616445101

Class p2
Precision = 0.8094313991849408
Recall = 0.826759167492567
F1_score = 0.818003530103942

Class p3
Precision = 0.33869670152855996
Recall = 0.3177358490566038
F1_score = 0.3278816199376947

Class p4
Precision = 0.647
Recall = 0.6287657920310982
F1_score = 0.637752587481518



In [ ]:

import optuna
from optuna.samplers import GridSampler

param_grid = {
    'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
    'learning_rate'   : [0.001, 0.01, 0.1, 1],
    'max_depth'       : [3, 5, 8, 10],
    'alpha'           : [1, 10, 100],
    'n_estimators'    : [10, 50, 100],
}

y = df_encoded['Approved_Flag']
X = df_encoded.drop(['Approved_Flag'], axis=1)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

answers_grid = {
    'combination'      : [],
    'train_Accuracy'   : [],
    'test_Accuracy'    : [],
    'colsample_bytree' : [],
    'learning_rate'    : [],
    'max_depth'        : [],
    'alpha'            : [],
    'n_estimators'     : []
}

\
sampler = GridSampler(search_space=param_grid)
study = optuna.create_study(direction="maximize", sampler=sampler)

def objective(trial: optuna.Trial) -> float:
   
    params = {
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', param_grid['colsample_bytree']),
        'learning_rate'   : trial.suggest_categorical('learning_rate',    param_grid['learning_rate']),
        'max_depth'       : trial.suggest_categorical('max_depth',        param_grid['max_depth']),
        'alpha'           : trial.suggest_categorical('alpha',            param_grid['alpha']),
        'n_estimators'    : trial.suggest_categorical('n_estimators',     param_grid['n_estimators']),
    }

    model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=4,
        **params
    )

    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test  = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc  = accuracy_score(y_test,  y_pred_test)

    trial.set_user_attr('train_accuracy', float(train_acc))
    trial.set_user_attr('test_accuracy',  float(test_acc))

    idx = trial.number + 1
    print(f"Combination {idx}")
    print(f"colsample_bytree: {params['colsample_bytree']}, "
          f"learning_rate: {params['learning_rate']}, "
          f"max_depth: {params['max_depth']}, "
          f"alpha: {params['alpha']}, "
          f"n_estimators: {params['n_estimators']}")
    print(f"Train Accuracy: {train_acc:.2f}")
    print(f"Test Accuracy : {test_acc :.2f}")
    print("-" * 30)

    return test_acc  

n_trials = np.prod([len(v) for v in param_grid.values()])
study.optimize(objective, n_trials=int(n_trials), show_progress_bar=True)


for i, t in enumerate(study.trials, start=1):
    answers_grid['combination']      .append(i)
    answers_grid['train_Accuracy']   .append(t.user_attrs['train_accuracy'])
    answers_grid['test_Accuracy']    .append(t.user_attrs['test_accuracy'])
    answers_grid['colsample_bytree'] .append(t.params['colsample_bytree'])
    answers_grid['learning_rate']    .append(t.params['learning_rate'])
    answers_grid['max_depth']        .append(t.params['max_depth'])
    answers_grid['alpha']            .append(t.params['alpha'])
    answers_grid['n_estimators']     .append(t.params['n_estimators'])

answers_df = pd.DataFrame(answers_grid)

print("Best Train Accuracy:", study.best_value)
print("Best Test Accuracy:", study.best_value)
print("Best Params:", study.best_params)


[I 2025-11-04 00:09:18,576] A new study created in memory with name: no-name-686db8ef-fe39-41c6-965c-9b158b74acbf


  0%|          | 0/720 [00:00<?, ?it/s]

Combination 1
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 10, alpha: 100, n_estimators: 100
Train Accuracy: 0.61
Test Accuracy : 0.61
------------------------------
[I 2025-11-04 00:09:19,440] Trial 0 finished with value: 0.6051349102579342 and parameters: {'colsample_bytree': 0.1, 'learning_rate': 0.01, 'max_depth': 10, 'alpha': 100, 'n_estimators': 100}. Best is trial 0 with value: 0.6051349102579342.
Combination 2
colsample_bytree: 0.7, learning_rate: 0.01, max_depth: 10, alpha: 100, n_estimators: 10
Train Accuracy: 0.74
Test Accuracy : 0.73
------------------------------
[I 2025-11-04 00:09:19,619] Trial 1 finished with value: 0.7337453940330441 and parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'alpha': 100, 'n_estimators': 10}. Best is trial 1 with value: 0.7337453940330441.
Combination 3
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 5, alpha: 10, n_estimators: 100
Train Accuracy: 0.61
Test Accuracy : 0.61
----------------------

In [ ]:
# Define the hyperparameter grid
param_grid = {
  'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
  'learning_rate'   : [0.001, 0.01, 0.1, 1],
  'max_depth'       : [3, 5, 8, 10],
  'alpha'           : [1, 10, 100],
  'n_estimators'    : [10,50,100]
}

index = 0

answers_grid = {
    'combination'       :[],
    'train_Accuracy'    :[],
    'test_Accuracy'     :[],
    'colsample_bytree'  :[],
    'learning_rate'     :[],
    'max_depth'         :[],
    'alpha'             :[],
    'n_estimators'      :[]

    }


# Loop through each combination of hyperparameters
for colsample_bytree in param_grid['colsample_bytree']:
  for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
      for alpha in param_grid['alpha']:
          for n_estimators in param_grid['n_estimators']:
             
              index = index + 1
             
              # Define and train the XGBoost model
              model = xgb.XGBClassifier(objective='multi:softmax',  
                                       num_class=4,
                                       colsample_bytree = colsample_bytree,
                                       learning_rate = learning_rate,
                                       max_depth = max_depth,
                                       alpha = alpha,
                                       n_estimators = n_estimators)
               
       
                     
              y = df_encoded['Approved_Flag']
              x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

              label_encoder = LabelEncoder()
              y_encoded = label_encoder.fit_transform(y)


              x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


              model.fit(x_train, y_train)
  

       
              # Predict on training and testing sets
              y_pred_train = model.predict(x_train)
              y_pred_test = model.predict(x_test)
       
       
              # Calculate train and test results
              
              train_accuracy =  accuracy_score (y_train, y_pred_train)
              test_accuracy  =  accuracy_score (y_test , y_pred_test)
              
              
       
              # Include into the lists
              answers_grid ['combination']   .append(index)
              answers_grid ['train_Accuracy']    .append(train_accuracy)
              answers_grid ['test_Accuracy']     .append(test_accuracy)
              answers_grid ['colsample_bytree']   .append(colsample_bytree)
              answers_grid ['learning_rate']      .append(learning_rate)
              answers_grid ['max_depth']          .append(max_depth)
              answers_grid ['alpha']              .append(alpha)
              answers_grid ['n_estimators']       .append(n_estimators)
       
       
              # Print results for this combination
              print(f"Combination {index}")
              print(f"colsample_bytree: {colsample_bytree}, learning_rate: {learning_rate}, max_depth: {max_depth}, alpha: {alpha}, n_estimators: {n_estimators}")
              print(f"Train Accuracy: {train_accuracy:.2f}")
              print(f"Test Accuracy : {test_accuracy :.2f}")
              print("-" * 30)



Combination 1
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 2
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 3
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 100
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 4
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 5
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 6
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 100
Train Accuracy: 0.61
Test Accu

In [50]:
# Convert answers_grid into a DataFrame
answers_df = pd.DataFrame(answers_grid)

# Find the row with the best (highest) test accuracy
best_row = answers_df.loc[answers_df['test_Accuracy'].idxmax()]

print("✅ Best Hyperparameter Combination:")
print(f"Combination #: {int(best_row['combination'])}")
print(f"Train Accuracy: {best_row['train_Accuracy']:.4f}")
print(f"Test Accuracy : {best_row['test_Accuracy']:.4f}")
print(f"colsample_bytree: {best_row['colsample_bytree']}")
print(f"learning_rate  : {best_row['learning_rate']}")
print(f"max_depth      : {int(best_row['max_depth'])}")
print(f"alpha          : {best_row['alpha']}")
print(f"n_estimators   : {int(best_row['n_estimators'])}")



✅ Best Hyperparameter Combination:
Combination #: 546
Train Accuracy: 0.8120
Test Accuracy : 0.7816
colsample_bytree: 0.7
learning_rate  : 1.0
max_depth      : 3
alpha          : 10.0
n_estimators   : 100
